In [ ]:
from numpy import *  #pi, sin, cos, sqrt, log, tan, atan
import numpy as np
from scipy.integrate import odeint 
from scipy.integrate import quad
def integrand(x):
    return x/sqrt(1-x**2)
def integrandn(x,n):
    return(x**(n+1)/sqrt(1-x**2))
def integrandn1(x,n):
    if abs(x)>0.002:
        y=x**n*(1.-1./sqrt(1.-x**2))
    else:
        y=-x**(n+2)*0.5*(1.+3.*x**2/4)
    return(y)
def E0_field(r,thet,fi,R0,Uloop):
    E0tor=Uloop/(2*pi*R0)
    return(E0tor)

def E_field(r,thet,fi,R0,E0tor):
    Etor=E0tor*R0/(R0+r*cos(thet))
    Erad=0.
    Epol=0.
    Etot=sqrt(Etor**2+Erad**2+Epol**2)
    if abs(Etot) >0.:
        etor=Etor/Etot
        erad=Erad/Etot
        epol=Epol/Etot
    else:
        etor=0.
        erad=0.
        epol=0.
    return(Etot,Etor,etor,Erad,erad,Epol,epol)

def Bpol_f(r,thet,sf,B0,R0):
    Bpol1=B0/(R0*sf)/(1+r/R0*cos(thet))
    Bpol=Bpol1*r
    return(Bpol,Bpol1)

def saf_fact(sf0,sfb,r,a):
    sf=sf0+(sfb-sf0)*(r/a)**2
    return(sf)

def fn(x,n):
    res1=0.
    if abs(x) >=5.e-2:
        for i in range(0,n):
            res1=res1+(-1)**i*x**(n-i)/(n-i)
        res1=res1+(-1)**n*log(abs(1.+x))
        res1=res1/(1+x)/x**(n+1)
    else:
        for i in range(0,10):
            res1=res1+(-1)**(n+i+2)*x**(+i)/(n+1+i)
        res1=res1*(-1)**n/(1+x)
    return(res1)
def Mag_field(r,thet,fi,R0,a,B0,delfi,nfi,delr,n,sf0,sfb):
    x=r/R0
    R=R0+r*cos(thet)
    xpr=r*cos(thet)/R0
    Fnpr=fn(xpr,n+1)
    Btor=B0*R0/R*(1.+delfi*cos(nfi*fi)*(1.+delr*cos(thet))*(r/a)**n)
    Gpr=B0*delfi*nfi*sin(nfi*fi)*(1.+delr*cos(thet))
    Gpr1=B0*delfi*nfi**2*cos(nfi*fi)*(1.+delr*cos(thet))
    Gpr2=-B0*delfi*nfi*sin(nfi*fi)*delr*sin(thet)
    Gpr3=Gpr*x*(r/a)**n
    Gpr31=Gpr*x*(r/a)**(n-1)/a
    Fpr=(r/a)**n*Fnpr
    if abs(xpr)>=0.01:  
        dFndthet=(Fnpr*((n+2)/xpr+1./(1.+xpr))-1./xpr/(1.+xpr)**2)*x*sin(thet)
    else:
        apr=0.
        bpr=0.
        for i in range(0,10):
            apr=+apr+(-1)**(n+1+i)*xpr**i/(n+2.+i)
        for i in range(1,10):
            bpr=bpr+(-1)**(n+1+i)*i*xpr**(i-1)/(n+2.+i)
        dFndthet=-((-1)**n/(1+xpr)**2*apr+(-1)**(n+1)/(1+xpr)*bpr)*x*sin(thet)    
#    print(xpr,dFndthet)
#    sys.exit()
    Brad1=Gpr*(1./R0)*Fpr
    Brad=r*Brad1
#    print('r,Brad',r,Brad,'Gpr,Fpr',Gpr,Fpr)
#######################################3333
#    poloidal field calculation
    psi0=2*pi*B0*R0**2
    psi0n=psi0*(R0/a)**n
#    psi0n1=psi0/R0/((n+1)*a**n)
#    print(psi0,psi0n,psi0n1)
#    A=psi0*(1-sqrt(1-x**2))
    res=quad(integrand,0.,r/R0)
#    print(res)
#    sys.exit()
    A1=psi0*res[0]
#    print(A,A1,A1/A,psi0)
#    print('A1',A1)
    resn=quad(integrandn,0.,r/R0,args=(n))
#    print('resn',resn)
    An=psi0n*resn[0]
#    print('An',An)
    resn1=quad(integrandn1,0.,r/R0,args=(n))
    An1=psi0n*resn1[0]
#    print('An1',An1)
    psitor=A1+(An+delr*An1)*delfi*cos(nfi*fi)
    A1=psitor
    rpsi=(R0/psi0)*sqrt((2*psi0-A1)*A1)
#    print('r,rpsi',r,rpsi)
    sf=saf_fact(sf0,sfb,rpsi,a)
#    sf1=saf_fact(sf0,sfb,r,a)
#    print("sf,sf1",sf,sf1)
    Bpol,Bpol1=Bpol_f(r,thet,sf,B0,R0)
#    print('Bpol',Bpol)
    
    dpsidA1=1.
    dA1dr=(psi0/R0)*x/sqrt(1-x**2)
    dpsidAn=delfi*cos(nfi*fi)
    dAndr=psi0n/R0*integrandn(x,n)     #*(n+1+x**2/(1.-x**2))
    dpsidAn1=delr*delfi*cos(nfi*fi)
#    print('psi0n1',psi0n1,n,r**(n-1))
    dAn1dr=psi0n*integrandn1(x,n)
    dpsidr=dpsidA1*dA1dr+dpsidAn*dAndr+dpsidAn1*dAn1dr
    dsfdrpsi=2.*(sfb-sf0)*rpsi/a**2
    dsfdr=dsfdrpsi*dpsidr
#    print('dpsidA1*dA1dr,    dpsidAn*dAndr,   dpsidAn1*dAn1dr,     dpsidr,      dsfdrpsi*dsfdr')
#    print(dpsidA1*dA1dr,dpsidAn*dAndr,dpsidAn1*dAn1dr,dpsidr,dsfdrpsi*dsfdr)
    
#    dBtordfinp
    ########################33
    Btot=sqrt(Btor**2+Bpol**2+Brad**2)
    brad=Brad/Btot
    bpol=Bpol/Btot
    bpol1=Bpol1/Btot
    btor=Btor/Btot
    
    dBtordr=-Btor*cos(thet)/R+B0*R0/R*delfi*cos(nfi*fi)*(1+delr*cos(thet))*(r/a)**(n-1)*n/a
    dBpoldr=(B0/R)/sf*(1.-xpr/(1.+xpr)-r*dsfdr/sf)
    dBtordfi=-B0/(1+xpr)*delfi*nfi*sin(nfi*fi)*(1.+delr*cos(thet))*(r/a)**n
    dBraddr=(-Brad1*(R0+2.*r*cos(thet))-dBtordfi)/R
#    dBpoldfi=-(Bpol/sf)*((sfb-sf0)/a**2)*(R0**2/psi0)*((An+delr*An1)*delfi*nfi*sin(nfi*fi))
    dBpoldfi=2.*(Bpol/sf)*((sfb-sf0)/a**2)*(R0**2/psi0**2*(psi0-psitor))*((An+delr*An1)*delfi*nfi*sin(nfi*fi))
#    dBpoldfi=2*B0*x/(1+xpr)/sf**2*(sfb-sf0)*(R0/a/psi0)**2*(psi0-psitor)*(delfi*An+delr*delfi*An1)*sin(nfi*fi)
#    dBpoldfi=2*B0*x/(1+xpr)/sf**2*(sfb-sf0)*(R0/a/psi0)**2*(psi0-psitor)*delfi*nfi*sin(nfi*fi)*An
    dBraddfi=Gpr1*(r/R0)*Fpr
    dBpoldthet1=Bpol*sin(thet)/R
    dBpoldthet=dBpoldthet1*r
#    dBtordthet1=Btor*r*sin(thet)/R0/(1.+xpr)-B0*delfi*cos(nfi*fi)*delr*sin(thet)/(1+xpr)*(r/a)**(n-1)/a
#    dBtordthet1=Btor*sin(thet)/R0/(1.+xpr)-B0*delfi*cos(nfi*fi)*delr*sin(thet)/(1+xpr)*(r/a)**(n-1)/a
    dBtordthet1=Btor*sin(thet)/R-B0*delfi*cos(nfi*fi)*delr*sin(thet)/(1+xpr)*(r/a)**(n-1)/a
    dBtordthet=dBtordthet1*r
#    dFndthet=[
    dBraddthet1=Gpr2*(1/R0)*Fpr+Gpr31*dFndthet
    dBraddthet=dBraddthet1*r
    
#    dBtotdr=Brad*dBraddr+Bpol*dBpoldr+Btor*dBtordr
#    dBtotdthet=Brad*dBraddthet+Bpol*dBpoldthet+Btor*dBtordthet
#    dBtotdfi=Brad*dBraddfi+Bpol*dBpoldfi+Btor*dBtordfi
#    print('dBraddfi=',dBraddfi,'dBpoldfi=',dBpoldfi,'dBtordfi=',dBtordfi,'dBtotdfi=',dBtotdfi)
#    print('Brad=',Brad,' Bpol=',Bpol,' Btor=',Btor)
#    print('sf=',sf)
#    print('Brad=',Brad,'dBraddr=',dBraddr,'dBraddthet=',dBraddthet,'dBraddfi=',dBraddfi)
#    sys.exit()

    return(R,Btot,Btor,Bpol,Bpol1,Brad,brad,btor,bpol,bpol1,dBpoldr,dBtordfi,dBraddr,dBtordr,dBpoldfi,dBraddfi,  \
    dBpoldthet,dBtordthet,dBraddthet,dBpoldthet1,dBtordthet1,dBraddthet1)
##############
#   dbrdthet=(dBraddthet-Brad*(Brad*dBraddthet+Bpol*dBpoldthet+Btor*dBtordthet)/Btot**2)/Btot
#   dbpoldr=(dBpoldr-Bpol*(Brad*dBraddr+Bpol*dBpoldr+Btor*dBtordr)/Btot**2)/Btot
#   rtbfi=(dbrdthet-bpol)/r-dbpoldr                                         
#   dbfidthet=(dBtordthet-Btor*(Brad*dBraddthet+Bpol*dBpoldthet+Btor*dBtordthet)/Btot**2)/Btot  

def rot_b(r,thet,fi,R,Btot,Btor,Bpol,Bpol1,Brad,brad,btor,bpol,bpol1,dBpoldr,dBtordfi,dBraddr,dBtordr,dBpoldfi,dBraddfi,  \
    dBpoldthet,dBtordthet,dBraddthet,dBpoldthet1,dBtordthet1,dBraddthet1):
    br=brad
    bpol=bpol
    bpol1=Bpol1/Btot
    bfi=btor

    dbrdthet1=(dBraddthet1-brad*(brad*dBraddthet1+bpol*dBpoldthet1+btor*dBtordthet1))/Btot
    dbrdthet=(dBraddthet-brad*(brad*dBraddthet+bpol*dBpoldthet+btor*dBtordthet))/Btot
    dbrdfi=(dBraddfi-brad*(brad*dBraddfi+bpol*dBpoldfi+btor*dBtordfi))/Btot

    dbpoldr=(dBpoldr-bpol*(brad*dBraddr+bpol*dBpoldr+btor*dBtordr))/Btot
    dbpoldfi=(dBpoldfi-bpol*(brad*dBraddfi+bpol*dBpoldfi+btor*dBtordfi))/Btot

    dbfidr=(dBtordr-btor*(brad*dBraddr+bpol*dBpoldr+btor*dBtordr))/Btot
    dbfidthet1=(dBtordthet1-btor*(brad*dBraddthet1+bpol*dBpoldthet1+btor*dBtordthet1))/Btot
    dbfidthet=(dBtordthet-btor*(brad*dBraddthet+bpol*dBpoldthet+btor*dBtordthet))/Btot
#    brtr=(dbpoldfi+bfi*sin(thet))/R-dbfidthet/r

    rtbr=(dbpoldfi+bfi*sin(thet))/R-dbfidthet1
    rtbpol=(bfi*cos(thet)-dbrdfi)/R+dbfidr
    rtbfi=(dbrdthet1-bpol1)-dbpoldr

    brtr=(bpol*rtbfi-bfi*rtbpol)
    brtt=(bfi*rtbr-br*rtbfi)
    brtfi=(br*rtbpol-bpol*rtbr)

    gbr=(Brad*dBraddr+Bpol*dBpoldr+Btor*dBtordr)/Btot
    gbt=(Brad*dBraddthet1+Bpol*dBpoldthet1+Btor*dBtordthet1)/Btot
    gbfi=(Brad*dBraddfi+Bpol*dBpoldfi+Btor*dBtordfi)/Btot/R

    glbr=gbr/Btot
    glbt=gbt/Btot
    glbfi=gbfi/Btot

    bgrr=bpol*glbfi-btor*glbt
    bgrt=btor*glbr-br*glbfi
#    bgrt1=btor*gbr1-br*gbfi1
    bgrfi=br*glbt-bpol*glbr

    bbrtr=bpol*brtfi-bfi*brtt
    bbrtt=bfi*brtr-br*brtfi
#bbrtt1=bfi*brtr-br*brtfi
    bbrtfi=br*brtt-bpol*brtr
    return(rtbr,rtbpol,rtbfi,brtr,brtt,brtfi,gbr,gbt,gbfi,bgrr,bgrt,bgrfi,bbrtr,bbrtt,bbrtfi)

def eq_mot(eqq,ccc,m0,R0,pperp,ppar,r,thet,fi,R,Uloop,brtr,brtt,brtfi,gbr,gbt,gbfi, \
    bgrr,bgrt,bgrfi,bbrtr,bbrtt,bbrtfi,brad,btor,bpol,muini,Btot,dBpoldr,dBpoldthet,dBpoldfi, \
    dBraddr,dBraddthet,dBraddfi,dBtordr,dBtordthet,dBtordfi,Bpol,Brad,Btor):
    E0tor=E0_field(r,thet,fi,R0,Uloop)
    Etot,Etor,etor,Erad,erad,Epol,epol=E_field(r,thet,fi,R0,E0tor)
    ptot2=pperp**2+ppar**2
    gam=sqrt(1.+ptot2)
#    dppardt=eqq*R0/(m0*ccc**2)*(Erad*brad+Epol*bpol+Etor*btor)-R0*pperp**2/(2*Btot)* \
    dppardt=eqq*R0/(m0*ccc**2)*(Erad*brad+Epol*bpol+Etor*btor)-R0*pperp**2/(2*Btot*gam)* \
    (gbr*brad+gbt*bpol+gbfi*btor)
#    (gbr*brad+gbt*bpol+gbfi*btor)
    omce=eqq*Btot/(m0*ccc)
    M1=ppar*R0/gam
    M2=0.5*R0/(omce*gam)*pperp**2
#    M2=2*R0/(omce*gam)*pperp**2
    M3=R0/(omce*gam)*ppar**2
    dRdtr=M1*brad+M2*bgrr+M3*bbrtr
    dRdtt=M1*bpol+M2*bgrt+M3*bbrtt
    dRdtfi=M1*btor+M2*bgrfi+M3*bbrtfi
    y1=dppardt
    y2=dRdtr
    y3=dRdtt/r
    y4=dRdtfi/R
    dpperp2dt=muini*(gbr*y2+gbt*y3*r+gbfi*y4*R)
#    print('dBtotdr=',dBtotdr,'dBtotdthet=',dBtotdthet,'dBtotdfi=',dBtotdfi)
#    sys.exit()
#    dpperp2dt=muini/Btot*(dBtotdr*y2+dBtotdthet*y3)
#    dpperp2dt=muini/Btot*(dBtotdr*y2+dBtotdfi*y4)
#    dpperp2dt=muini/Btot*(dBtotdthet*y3+dBtotdfi*y4)
#    dpperp2dt=muini/Btot*(dBtotdfi*y4)
    y5=dpperp2dt
    dBpoldt=dBpoldr*y2+dBpoldthet*y3+dBpoldfi*y4
    y6=dBpoldt
    dBtotdt=gbr*y2+gbt*y3*r+gbfi*y4*R
    y7=dBtotdt
    dBraddt=dBraddr*y2+dBraddthet*y3+dBraddfi*y4
#    print('Brad=',Brad,'dBraddr=',dBraddr,'dBraddthet=',dBraddthet,'dBraddfi=',dBraddfi)
#    sys.exit()
    y8=dBraddt
    dBtordt=dBtordr*y2+dBtordthet*y3+dBtordfi*y4
    y9=dBtordt
    dydt=[y1,y2,y3,y4,y5,y6,y7,y8,y9]
    return dydt

def fin_fun(t, y,eqq,m0,ccc,a,R0,B0,sf0,sfb,delr,delfi,nfi,Uloop,n,pparini,pperpini,muini):
    ppar=y[0]
    r=y[1]
    thet=y[2]
    fi=y[3]
    pperp2pr=y[4]
    pperppr=sqrt(pperp2pr)
    Bpol=y[5]
    Btot=y[6]
    Brad=y[7]
    Btor=y[8]
    sf=saf_fact(sf0,sfb,r,a)
    Bpol,Bpol1=Bpol_f(r,thet,sf,B0,R0)
#    E0tor=E0_field(r,thet,fi,R0,Uloop)
#    Etot,Etor,etor,Erad,erad,Epol,epol=E_field(r,thet,fi,R0,E0tor)
    R,Btot,Btor,Bpol,Bpol1,Brad,brad,btor,bpol,bpol1,dBpoldr,dBtordfi,dBraddr,dBtordr,dBpoldfi,dBraddfi,  \
    dBpoldthet,dBtordthet,dBraddthet,dBpoldthet1,dBtordthet1,dBraddthet1 \
    =Mag_field(r,thet,fi,R0,a,B0,delfi,nfi,delr,n,sf0,sfb)
    rtbr,rtbpol,rtbfi,brtr,brtt,brtfi,gbr,gbt,gbfi,bgrr,bgrt,bgrfi,bbrtr,bbrtt,bbrtfi  \
    =rot_b(r,thet,fi,R,Btot,Btor,Bpol,Bpol1,Brad,brad,btor,bpol,bpol1,dBpoldr,   \
    dBtordfi,dBraddr,dBtordr,dBpoldfi,dBraddfi,  \
    dBpoldthet,dBtordthet,dBraddthet,dBpoldthet1,dBtordthet1,dBraddthet1)
    pperp=sqrt(muini*Btot)
    dydt=eq_mot(eqq,ccc,m0,R0,pperp,ppar,r,thet,fi,R,Uloop,brtr,brtt,brtfi,gbr,gbt,gbfi, \
    bgrr,bgrt,bgrfi,bbrtr,bbrtt,bbrtfi,brad,btor,bpol,muini,Btot,dBpoldr,dBpoldthet,dBpoldfi,  \
    dBraddr,dBraddthet,dBraddfi,dBtordr,dBtordthet,dBtordfi,Bpol,Brad,Btor)
    return dydt 
m01=9.1094e-28
ccc1=3.e10
m0=9.1094e-31
eqq=1.602e-19  
ccc=3.e8        
a=0.075
R0=0.55
B0=2.2
sf0=1.
sfb=3.2
delr=0.95
delfi=0.1
nfi=14
Uloop=00000.    #Volts
n=4  # power in dependence of toroidal field ripple on (r/a)**n 
pparini=21.0
pperpini=0.1
rini=0.01
thetini=pi
fiini=0.
sf=saf_fact(sf0,sfb,rini,a)
Bpol,Bpol1=Bpol_f(rini,thetini,sf,B0,R0)    
R,Btot,Btor,Bpol,Bpol1,Brad,brad,btor,bpol,bpol1,dBpoldr,dBtordfi,dBraddr,dBtordr,dBpoldfi,dBraddfi,  \
dBpoldthet,dBtordthet,dBraddthet,dBpoldthet1,dBtordthet1,dBraddthet1 \
=Mag_field(rini,thetini,fiini,R0,a,B0,delfi,nfi,delr,n,sf0,sfb)
muini=pperpini**2/Btot
p2ini=pparini**2+pperpini**2
#print('Btot=',Btot,'sqrt(Brad**2+Bpol**2+Btor**2)=',sqrt(Brad**2+Bpol**2+Btor**2))
#print('Brad=',Brad,'Bpol=',Bpol,'Btor=',Btor)
#print('muini=',muini,'pperpini**2=',pperpini**2,'Btot=',Btot)
#sys.exit()

#sol=solve_ivp(fin_fun, [0,150], y0, method='RK45', t, \
#sol=RK45(fin_fun, [0,150], y0,t, vectorised=False,  \
#args=(eqq,m0,ccc,a,R0,B0,sf0,sfb,delr,delfi,nfi,Uloop,n,pparini,pperpini,muini),dense_output=True)

#print(t[1],t[10],t[50])
#print(sol[0,0],sol[10,0],sol[50,0])
#print(sol[:,1])
#print(sol[0,1],sol[10,1],sol[50,1])
#print(sol[0,2],sol[10,2],sol[50,2])
#print(sol[0,3],sol[10,3],sol[50,3])



In [ ]:
R0= 0.55
B0= 2.2

#t = np.linspace(0, 1, 3)
#print(t)
#sys.exit()
y0=[pparini,rini,thetini,fiini,pperpini**2,Bpol,Btot,Brad,Btor]
#from scipy.integrate.solve_ivp import odeint
from scipy.integrate import odeint,solve_ivp
#sol = odeint(count_fin_fun,y0,t,args=(eqq,m0,ccc,a,R0,B0,sf0,sfb,delr,delfi,nfi,Uloop,n,pparini,pperpini,muini))
sol= solve_ivp(fin_fun,  [0, 5000], y0, method='LSODA', args=(eqq,m0,ccc,a,R0,B0,sf0,sfb,delr,delfi,nfi,Uloop,n,pparini,pperpini,muini),
               rtol = 1e-6, atol= 1e-9)
sol

In [ ]:

import matplotlib.pyplot as plt
plt.plot(sol.t, sol.y[0], 'b', label='ppar(t)')
plt.plot(sol.t, sqrt(p2ini-muini*sol.y[6]), 'g', label='pparcalc(t)')
plt.plot(sol.t, sqrt(p2ini-sol.y[4]), 'y', label='pparcalc(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, (sol.y[0]-sqrt(p2ini-muini*sol.y[6]))/sol.y[0], 'b', label='(ppar(t)-pparcalc(t))/ppar(t)')
plt.plot(sol.t, (sol.y[0]-sqrt(p2ini-sol.y[4]))/sol.y[0], 'g', label='(ppar(t)-sqrt(p2ini-pperp2pr(t))/ppar(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, sol.y[4], 'b', label='pperp2pr(t)')
plt.plot(sol.t, p2ini-sol.y[0]**2, 'g', label='pperp2calc(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()


In [ ]:
plt.plot(sol.t, (sol.y[4]-(p2ini-sol.y[0]**2))/sol.y[4], 'b', label='(pperp2pr-pperp2calc)/pperp2(t)')

plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()


In [ ]:
plt.plot(sol.t, sol.y[1]/a, 'g', label='r(t)/a')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()


In [ ]:

plt.plot(sol.t, sol.y[2], 'g', label='thet(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()


In [ ]:

plt.plot(sol.t, sol.y[3], 'g', label='fi(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()


In [ ]:

plt.plot(sol.t, sol.y[5], 'g', label='Bpol(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, sol.y[7], 'g', label='Brad(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, sol.y[8], 'g', label='Btor(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, sol.y[6], 'g', label='Btot(t)')
plt.plot(sol.t, sqrt((sol.y[5])**2+(sol.y[7])**2+(sol.y[8])**2), 'b', label='Btotcalc(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:

plt.plot(sol.t,sol.y[6]-sqrt((sol.y[5])**2+(sol.y[7])**2+(sol.y[8])**2), 'b', label='del(Btotcalc(t))')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t,(sol.y[4])/(sol.y[6]), 'g', label='pperp^2/Btot(t)')
#plt.plot(t,(sol[:, 4])/(sol[:,6])+1.e-5, 'g', label='pperp^2/Btot(t)')
plt.plot(sol.t, (p2ini-sol.y[0]**2)/(sol.y[6]), 'r', label='pperp2calc(t)**2/Btot(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, ((sol.y[4])+(sol.y[0])**2)/p2ini, 'g', label='(pperp^2+ppar^2)/p2ini')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:

plt.plot(sol.t, m01*ccc1**2*(sqrt(1+(sol.y[4])+(sol.y[0])**2)-1.)/1.6022e-12, 'r', label='energy(keV)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(sol.t, (sqrt(1+(sol.y[4])+(sol.y[0])**2)-1.)/(sqrt(1+(sol.y[4])+(sol.y[0][0])**2)-1.),  \
'g', label='relative del(energy)')
#plt.plot(t, 512.*(sqrt(1+(sol[:, 4])+(sol[:,0])**2)-1.), 'r', label='energy(keV)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

r = sol.y[1]/a
theta = sol.y[2]

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, r)
ax.set_rmax(1)
ax.set_rticks([0.2, 0.4, 0.6, 0.8])  # Less radial ticks
ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True)

ax.set_title("Electron trajectory in poloidal crossection", va='bottom')
plt.show()

In [ ]:

def myfun(x,a,b):
    return(1./(a+b*cos(x)))
r=0.000001
R0=0.55
a=1.
b=r/R0
thet1=pi
thet0=0.
res0=4/sqrt(a**2-b**2)*(atan((a-b)*tan(thet1/2)/sqrt(a**2-b**2))-atan((a-b)*tan(thet0/2)/sqrt(a**2-b**2)))
res1=4/sqrt(1.-b**2)*(atan((1.-b)*tan(pi/2.)/sqrt(1.-b**2)))
print(res0,res1)
res2=quad(myfun,0,2.*pi,args=(a,b))
print(res2[0])

In [ ]:
x=0.002
y=1.-1./sqrt(1.-x**2)
z=-x**2*0.5*(1+3.*x**2/4)
print(y,z)

In [ ]:
def fn(x,n):
    res1=0.
    if abs(x) >=5.e-2:
        for i in range(0,n):
            res1=res1+(-1)**i*x**(n-i)/(n-i)
        res1=res1+(-1)**n*log(abs(1.+x))
        res1=res1/(1+x)/x**(n+1)
    else:
        for i in range(0,10):
            res1=res1+(-1)**(n+i+2)*x**(+i)/(n+1+i)
        res1=res1*(-1)**n/(1+x)
    return(res1)
R0=0.55
a=0.075
r=0.007
x=r/R0
thet=pi/3
fi=pi/3
R=R0+r*cos(thet)
xpr=r*cos(thet)/R0
Fnpr=fn(xpr,n+1)
Btor=B0*R0/R*(1.+delfi*cos(nfi*fi)*(1.+delr*cos(thet))*(r/a)**n)
Gpr=B0*delfi*nfi*sin(nfi*fi)*(1.+delr*cos(thet))
Gpr1=B0*delfi*nfi**2*cos(nfi*fi)*(1.+delr*cos(thet))
Gpr2=-B0*delfi*nfi*sin(nfi*fi)*delr*sin(thet)
Gpr3=Gpr*x*(r/a)**n
Gpr31=Gpr*x*(r/a)**(n-1)/a
Fpr=(r/a)**n*Fnpr
#if abs(xpr)>=0.01:  
dFndthet=(Fnpr*((n+2)/xpr+1./(1.+xpr))-1./xpr/(1.+xpr)**2)*x*sin(thet)
#else:
apr=0.
bpr=0.
for i in range(0,10):
    apr=+apr+(-1)**(n+1+i)*xpr**i/(n+2.+i)
for i in range(1,10):
    bpr=bpr+(-1)**(n+1+i)*i*xpr**(i-1)/(n+2.+i)
dFndthet1=-((-1)**n/(1+xpr)**2*apr+(-1)**(n+1)/(1+xpr)*bpr)*x*sin(thet)        
print(xpr,dFndthet,dFndthet1)
